# Лабораторна робота №3

**Тема:** Класифікація

## Мета роботи
Закріпити основні поняття використання моделі  класифікації в машинному навчанні.

Завдання:

1.	Ознайомитеся з набором даних, що містить вимірювання, отримані з зображень двох видів турецького рису.

2.	Створіть бінарний класифікатор для розподілу зерен рису на два види.


3.	Оцініть ефективність моделі.




**Хід роботи**

Для виконання завдання будемо використовувати файл з даними, який доступный за посиланням: https://www.kaggle.com/datasets/muratkokludataset/rice-dataset-commeo-and-osmancik

В ньому розташовані дані про 2 види рису Commeo and Osmancik

Перегляньте, яка інформація там є:

Area: повертає кількість пікселів у межах рисового зерна.

Perimeter: обчислює окружність шляхом обчислення відстані між пікселями навколо меж рисового зерна.

Major Axis Length: це найдовша лінія, яку можна накреслити на рисовому зерні, тобто відстань головної осі.

Minor Axis Length: дає найкоротша лінія, яку можна намалювати на рисовому зерні, тобто мала відстань по осі.

Eccentricity: він вимірює, наскільки округлим є еліпс, який має ті самі моменти, що й рисове зерно.

Convex Area: повертає кількість пікселів найменшої опуклої оболонки області, утвореної рисовим зерном.

Extent: повертає співвідношення області, утвореної рисовим зерном, до пікселів обмежувальної рамки

Класи: Коммео і Османчик



**Крок 1.** Імпортуйте необхідні бібліотеки

In [ ]:
# Робота з даними
import pandas as pd
import numpy as np

# Візуалізація
import matplotlib.pyplot as plt
import seaborn as sns

# Машинне навчання
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Класифікаційні моделі
from sklearn.neighbors import KNeighborsClassifier    # KNN
from sklearn.tree import DecisionTreeClassifier, plot_tree # дерево рішень
from sklearn.naive_bayes import GaussianNB            # наївний Байєс

# Метрики оцінки якості
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    roc_curve,
    roc_auc_score
)

**Крок 2**. Завантажте дані з хмари

In [ ]:
# Пряме посилання на датасет (Cammeo vs Osmancik)
df = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/Rice_Cammeo_Osmancik.csv")

# Перевірка
print(df.shape)
print(df.head())
print(df['Class'].value_counts())

**Крок 3.** Підготуємо набори даних для класифікації

Виділемо ознаки та ціль: X — числові характеристики зерна, y — клас (Cammeo чи Osmancik).

Розділемо дані: тренувальні (80%) для навчання, тестові (20%) для перевірки.

Відмасштабуємо ознаки: щоб усі параметри були в одному діапазоні (важливо для логістичної регресії та KNN).

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Вибираємо ознаки (усі числові стовпці, крім 'Class')
X = df.drop(columns=['Class'])
y = df['Class']   # цільова змінна: Cammeo / Osmancik

# 2. Розділення на тренувальні та тестові дані
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,        # 20% даних для тесту
    random_state=42,      # фіксуємо випадковість
    stratify=y            # зберігаємо пропорцію класів
)

# 3. Масштабування ознак (рекомендовано для лінійних моделей)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Розмір тренувальної вибірки:", X_train.shape)
print("Розмір тестової вибірки:", X_test.shape)

**Крок 4. Класифікація за моделлю KNN**

Почнемо з найпростішого класифікатора — K‑Nearest Neighbors (KNN). Він працює за принципом: новий об’єкт відноситься до того класу, якому належить більшість його найближчих сусідів.

Пояснення
n_neighbors=5 → модель дивиться на 5 найближчих сусідів і відносить об’єкт до більшості.

Accuracy → частка правильних передбачень.

Classification report → precision, recall, F1 для кожного класу.

Confusion matrix → показує, де саме модель плутає Cammeo та Osmancik.

In [ ]:
# 1. Створюємо модель KNN
knn = KNeighborsClassifier(n_neighbors=5)  # параметр k = 5

# 2. Навчаємо модель на тренувальних даних
knn.fit(X_train_scaled, y_train)

# 3. Робимо прогноз на тестових даних
y_pred_knn = knn.predict(X_test_scaled)

# 4. Оцінюємо якість
print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("\nClassification report:\n", classification_report(y_test, y_pred_knn))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred_knn))

**Інтерпретація**
Accuracy = 0.9094. Accuracy показує частку правильних передбачень серед усіх прикладів.У нашому випадку модель правильно класифікувала ~91% зерен рису.
Це досить високий показник, але він не завжди відображає баланс між класами (тому дивимося на precision/recall).

Для класу Cammeo:

Precision = 0.91 → серед усіх зерен, які модель позначила як Cammeo, 91% справді Cammeo.

Recall = 0.88 → модель знайшла 88% усіх Cammeo, але 12% пропустила (помилково віднесла до Osmancik).

F1 = 0.89 → збалансований показник точності та повноти.

Для класу Osmancik:

Precision = 0.91 → серед усіх передбачених Osmancik, 91% справді Osmancik.

Recall = 0.93 → модель знайшла 93% усіх Osmancik, лише 7% пропустила.

F1 = 0.92 → дуже хороший баланс.

Загалом:

Macro avg = 0.91 → середнє значення метрик для обох класів.

Weighted avg = 0.91 → середнє з урахуванням кількості прикладів у кожному класі.

Це означає, що модель працює стабільно для обох сортів рису.

Confusion Matrix:

Рядок 1 (Cammeo):287 → правильно класифіковані як Cammeo.39 → помилково класифіковані як Osmancik.

Рядок 2 (Osmancik):30 → помилково класифіковані як Cammeo.406 → правильно класифіковані як Osmancik.

Висновок: модель трохи частіше плутає Cammeo з Osmancik, ніж навпаки.

**Крок 5. Дерево рішень**
Побудуємо ту ж саму класифікацію іншим методом.

DecisionTreeClassifier будує дерево правил для класифікації. Воно інтуїтивно зрозуміле: будує правила типу «якщо довжина > 10 → Osmancik».

max_depth=5 → обмежує складність дерева, щоб уникнути перенавчання.


In [ ]:
# 1. Створюємо модель дерева рішень
dt = DecisionTreeClassifier(random_state=42, max_depth=5)  # обмежимо глибину для простоти

# 2. Навчаємо модель
dt.fit(X_train_scaled, y_train)

# 3. Прогнозуємо на тестових даних
y_pred_dt = dt.predict(X_test_scaled)

# 4. Оцінка якості
print("Accuracy:", accuracy_score(y_test, y_pred_dt))
print("\nClassification report:\n", classification_report(y_test, y_pred_dt))
print("\nConfusion matrix:\n", confusion_matrix(y_test, y_pred_dt))

Зробіть інтерпретацію результатів моделювання самостійно:


**Крок 6**.Візуалізуємо дерево рішень

In [ ]:
# Створюємо та навчаємо дерево рішень
dt = DecisionTreeClassifier(random_state=42, max_depth=3)  # обмежимо глибину для читабельності
dt.fit(X_train_scaled, y_train)

# Візуалізація дерева
plt.figure(figsize=(16,10))
plot_tree(
    dt,
    feature_names=X.columns,      # назви ознак
    class_names=dt.classes_,      # назви класів (Cammeo, Osmancik)
    filled=True,                  # кольорове заповнення вузлів
    rounded=True,                 # округлені вузли
    fontsize=10
)

**Крок 7**

Вибір оптимального значення параметра k (для KNN)

Оскільки KNN залежить від параметра k (n_neighbors), його випадковий вибір (k=5) може бути неоптимальним. Ми знайдемо найкраще k за допомогою перебору.

In [ ]:
# Перевірка k від 1 до 20
k_values = range(1, 21)
accuracy_scores = []

for k in k_values:
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X_train_scaled, y_train)
    y_pred = knn_model.predict(X_test_scaled)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

# Знаходимо найкраще k
best_k = k_values[np.argmax(accuracy_scores)]
best_accuracy = np.max(accuracy_scores)

print(f"\n--- Вибір оптимального k для KNN ---")
print(f"Найвища точність (Accuracy): {best_accuracy:.4f} при k = {best_k}")

# Візуалізація залежності точності від k
plt.figure(figsize=(10, 6))
plt.plot(k_values, accuracy_scores, marker='o', linestyle='--')
plt.title('Точність KNN залежно від значення k')
plt.xlabel('Значення K')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()


Тут мають бути Ваші висновки

**Крок 10** Оцінка значущості ознак (Для Decision Tree)

Decision Tree, будучи деревною моделлю, може прямо оцінювати, які ознаки були найважливішими для прийняття рішень. Це властивість feature_importances_.

In [ ]:
# Використовуємо навчену модель dt з Кроку 5
feature_importances = dt.feature_importances_
feature_names = X.columns

# Створюємо DataFrame для зручного відображення
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)

print("\n--- Оцінка значущості ознак (Decision Tree) ---")
print(importance_df)

# Візуалізація важливості ознак
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Важливість ознак для класифікації рису')
plt.show()

Ваші висновки тут:

Спробуйте побудувати модель логістичної регресії для цієї задачі. Оцініть якість моделі. Використайте метод регуляризації.

Підказка: Використовуйте  

model = LogisticRegression(penalty='l2', C=1.0, solver='lbfgs', max_iter=1000)

де

LogisticRegression — виклик моделі логістичної регресії зі sklearn.

penalty='l2' означає, що використовується L2-регуляризація (штраф за великі коефіцієнти).

C=1.0  Це параметр сили регуляризації.Велике С - слабка регуляризація (модель може мати великі коефіцієнти). Маленьке 𝐶 - сильна регуляризація (ваги притискаються до нуля). Значення 1.0 — стандартний баланс.

solver='lbfgs' Це алгоритм оптимізації, який використовується для знаходження коефіцієнтів.

max_iter=1000  Максимальна кількість ітерацій для алгоритму оптимізації.

In [ ]:
Ваш код:

Ваші висновки: